In [1]:
import pymysql
import pandas as pd
import numpy as np
import scipy.stats as stats
import logging

# Configuración del logging
logging.basicConfig(
    level=logging.INFO,  # Nivel de log (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler(),  # Muestra los logs en la consola
        logging.FileHandler("etl_process.log")  # Guarda los logs en un archivo
    ]
)

# Configuración de la conexión
endpoint = "mat-trn-src.co4vks4ks8wo.eu-west-1.rds.amazonaws.com"
port = 3306
database = "training"
username = "srcuser"
password = "matillion"

# Conectar a la base de datos y cargar los datos en un DataFrame
try:
    logging.info("Estableciendo conexión con la base de datos.")
    connection = pymysql.connect(
        host=endpoint,
        user=username,
        password=password,
        database=database,
        port=port
    )
    
    # Crear un cursor para ejecutar la consulta
    with connection.cursor() as cursor:
        logging.info("Ejecutando consulta SQL.")
        sql_query = """
        SELECT `iata`, `airport`, `city`, `state`, `country`, `lat`, `long`, `num_carriers`, `is_active`
        FROM training_rds_airports;
        """
        cursor.execute(sql_query)
        
        # Obtener los resultados
        results = cursor.fetchall()
        logging.info("Consulta ejecutada exitosamente y resultados obtenidos.")
        
        # Crear un DataFrame a partir de los resultados
        columns = ['iata', 'airport', 'city', 'state', 'country', 'lat', 'long', 'num_carriers', 'is_active']
        df = pd.DataFrame(results, columns=columns)
        logging.info("DataFrame creado a partir de los resultados.")

        # Paso 1: Transformar la columna 'is_active' de binario a booleano
        df['is_active'] = df['is_active'].apply(lambda x: x == b'\x01')
        logging.info("Columna 'is_active' transformada a booleano.")
        
        # Análisis de cardinalidad para las variables categóricas
        categorical_columns = ['iata', 'airport', 'city', 'state', 'country']
        logging.info("Analizando la cardinalidad de las variables categóricas.")
        for column in categorical_columns:
            unique_values = df[column].nunique()
            logging.info(f"{column}: {unique_values} categorías únicas.")

        # Verificar normalidad de la columna 'num_carriers'
        logging.info("Verificando la normalidad de la columna 'num_carriers'.")

        # Prueba de Shapiro-Wilk
        shapiro_stat, shapiro_p_value = stats.shapiro(df['num_carriers'])
        logging.info(f"Prueba de Shapiro-Wilk completada: Estadístico={shapiro_stat}, p-valor={shapiro_p_value}")
        
        # Prueba de Kolmogorov-Smirnov
        ks_stat, ks_p_value = stats.kstest(df['num_carriers'], 'norm', args=(df['num_carriers'].mean(), df['num_carriers'].std()))
        logging.info(f"Prueba de Kolmogorov-Smirnov completada: Estadístico={ks_stat}, p-valor={ks_p_value}")

        # Interpretación de resultados
        alpha = 0.05
        if shapiro_p_value > alpha and ks_p_value > alpha:
            logging.info("La columna 'num_carriers' sigue una distribución normal.")
        else:
            logging.info("La columna 'num_carriers' no sigue una distribución normal.")

        # Paso 2: Aplicar One-Hot Encoding a las variables categóricas
        df_one_hot = pd.get_dummies(df, columns=categorical_columns)
        logging.info("One-Hot Encoding aplicado a las variables categóricas.")
        
        # Paso 3: Aplicar Frequency Encoding a las mismas variables categóricas
        df_freq = df.copy()  # Crear una copia para no sobrescribir el original
        for column in categorical_columns:
            freq_encoding = df_freq[column].value_counts() / len(df_freq)
            df_freq[column + '_freq'] = df_freq[column].map(freq_encoding)

        df_freq = df_freq.drop(columns=categorical_columns)
        logging.info("Frequency Encoding aplicado a las variables categóricas.")

        # Paso 4: Aplicar Target Encoding a las variables categóricas
        np.random.seed(0)
        df['target_value'] = np.random.randint(0, 100, size=len(df))
        logging.info("Columna de target simulada agregada para Target Encoding.")
        
        df_target = df.copy()  # Crear una copia para aplicar Target Encoding
        for column in categorical_columns:
            target_mean = df_target.groupby(column)['target_value'].mean()
            df_target[column + '_target'] = df_target[column].map(target_mean)

        df_target = df_target.drop(columns=categorical_columns)
        logging.info("Target Encoding aplicado a las variables categóricas.")

except pymysql.MySQLError as e:
    logging.error(f"Error al conectar o ejecutar la consulta: {e}")
finally:
    # Cerrar la conexión
    if connection:
        connection.close()
        logging.info("Conexión con la base de datos cerrada.")


2024-10-30 19:12:17,453 - INFO - Estableciendo conexión con la base de datos.
2024-10-30 19:12:18,329 - INFO - Ejecutando consulta SQL.
2024-10-30 19:12:19,109 - INFO - Consulta ejecutada exitosamente y resultados obtenidos.
2024-10-30 19:12:19,114 - INFO - DataFrame creado a partir de los resultados.
2024-10-30 19:12:19,117 - INFO - Columna 'is_active' transformada a booleano.
2024-10-30 19:12:19,119 - INFO - Analizando la cardinalidad de las variables categóricas.
2024-10-30 19:12:19,121 - INFO - iata: 3372 categorías únicas.
2024-10-30 19:12:19,122 - INFO - airport: 3233 categorías únicas.
2024-10-30 19:12:19,124 - INFO - city: 2672 categorías únicas.
2024-10-30 19:12:19,125 - INFO - state: 56 categorías únicas.
2024-10-30 19:12:19,126 - INFO - country: 5 categorías únicas.
2024-10-30 19:12:19,127 - INFO - Verificando la normalidad de la columna 'num_carriers'.
2024-10-30 19:12:19,130 - INFO - Prueba de Shapiro-Wilk completada: Estadístico=0.9564688446927009, p-valor=1.2614164683348